# testing poverty rate correlations

https://www.census.gov/data/datasets/2019/demo/saipe/2019-state-and-county.html

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px # choropleth maps
from IPython.display import Image
import os
import sys
import json
# plotly maps
# import plotly.figure_factory as ff
# from urllib.request import urlopen

# sys.path.append(os.path.abspath("C:/Users/Alex/Documents/My Data Files/COVID Research/analysis/mention_2_hashtag.py"))

# from analysis.functions.mention_2_hashtag import * 
# from analysis.functions.datamanip import *

# n grams
from sklearn.feature_extraction.text import CountVectorizer

raw = pd.read_csv('flair.joined.tweets.csv')

# changing date to more readable format
raw['created_at'] = pd.to_datetime(raw['created_at'])

# getting seperated date and time columns
raw['date'] = raw['created_at'].dt.date
raw['week'] = raw['created_at'].dt.week
raw['time'] = raw['created_at'].dt.time
raw['hour'] = raw['created_at'].dt.hour

min_wk = raw['week'].min()
max_wk = raw['week'].max()

min_date = raw['date'].min()
max_date = raw['date'].max()

# changing na to None
raw = raw.fillna('None')

raw = raw[raw['sentiment'] != 'None']

raw['is_negative'] = [1 if 'NEGATIVE' in str(sent) else 0 for sent in raw['sentiment']]
raw['is_positive'] = [1 if 'POSITIVE' in str(sent) else 0 for sent in raw['sentiment']]


def convert_fips(unknown_fips_list, fin):
    
    def convert(fipsline):
        name, postal, fips= fipsline.strip().split('\t')

        return {fips: name}

    fips_dict = {f: n 
                for dic in [convert(line)for line in open(fin, 'r')]
                for f, n in dic.items()}

    return [fips_dict[f'{unknown:02d}'] for unknown in unknown_fips_list]

raw = raw[raw['statefips'] != 'None']

raw['state_name'] = convert_fips([int(float(fips)) for fips in raw['statefips']],
                     '../analysis/functions/state.fips.txt')

us_state_abbrev = json.load(open('state_abbreviations.json'))

raw['state_abv'] = [us_state_abbrev[s] for s in raw['state_name']]

raw['fips'] = [f'{int(float(state)):02d}{int(float(county)):03d}' for i, state, county in raw[['statefips', 'countyfips']].itertuples()]
raw['count'] = 1
# raw.head()

all_splits = pd.to_datetime(['2020-03-05', '2020-03-26', '2020-05-01'])
raw['stage'] = ['s:1' if date < all_splits[0] else 's:2' if date < all_splits[1] else 's:3' if date < all_splits[2] else 's:4' for date in raw['date']]



#########################
# thresholds
thresholds = [0.8, 0.9, 0.95]
for thresh in thresholds:
    raw[f't:{thresh}'] = [1 if float(conf) > thresh else 0 for conf in raw['sent_confidence']]

col = 't:0.8'
raw['category'] = ['NEUT' if t == 0 else 'POS' if p == 1 else 'NEG' for i, n, p, t in raw[['is_negative', 'is_positive', col]].itertuples()]
raw['is_neutral'] = [1 if cat == 'NEUT' else 0 for cat in raw['category']]
raw['is_positive'] = [1 if cat == 'POS' else 0 for cat in raw['category']]
raw['is_negative'] = [1 if cat == 'NEG' else 0 for cat in raw['category']]

raw['is_neutral2'] = raw['is_neutral']
raw['is_positive2'] = raw['is_positive']
raw['is_negative2'] = raw['is_negative']

raw['is_neutral3'] = raw['is_neutral']
raw['is_positive3'] = raw['is_positive']
raw['is_negative3'] = raw['is_negative']

##################
# cdc

keywords = ['social distancing', 'social distance', 'physical distance', '6 feet', 'stay at home', 'school isolation', 'isolation', 'stay home', 'avoid touching', 'mask', 'covering', 'face shield', 'wear a mask', 'surgical mask', 'N95 respirator', 'wearing gloves', 'face shields', 'facial covering', 'skin protection', 'eye protection', 'ppe', 'wash hands', 'hand sanitizer', 'disinfect', 'clean', 'detergent', 'handwashing', 'hand hygiene', 'prevention hygiene', 'sprays', 'concentrates', 'wipes', 'routine cleaning', 'bleach solution', 'test', 'business closure']

topics = []
indexes = {i:0 for i in range(len(raw.index))}

for i, topic in enumerate(keywords):
    topics.append([1 if topic in text else 0 for text in raw['ogtext']])

for i, kw in enumerate(topics):
    for data_ind, value in enumerate(topics[i]):
        indexes[data_ind] += value

raw['in_cdc'] = [indexes[i] if 0 <= indexes[i] <= 1 else 1 for i in indexes]

cdc = raw[raw['in_cdc'] == 1]

cdc_splits = pd.to_datetime(['2020-03-12', '2020-04-20', '2020-05-14'])
cdc['stage'] = ['s:1' if date < cdc_splits[0] else 's:2' if date < cdc_splits[1] else 's:3' if date < cdc_splits[2] else 's:4' for date in cdc['date']]

# print(f'there are a total of {len(cdc.index)} tweets after subsetting for cdc keywords')
# print(raw)
# print(cdc)
print('loaded...')

loaded...


In [2]:
poverty = pd.read_csv('fips_poverty_est.csv')
poverty = poverty[['State FIPS Code', 'County FIPS Code', 'Poverty Percent, All Ages']]
poverty['statefips'] = [str(n).zfill(2) for n in poverty['State FIPS Code']]
poverty['countyfips'] = [str(n).zfill(3) for n in poverty['County FIPS Code']]
poverty['fips'] = [f'{s}{c}' for i,s,c in poverty[['statefips', 'countyfips']].itertuples()]
poverty = poverty[poverty['fips'].isin(raw['fips'].unique())]
poverty = poverty[['fips', 'Poverty Percent, All Ages']]
poverty.columns = ['fips', 'poverty_estimate']
poverty['poverty_estimate'] = poverty['poverty_estimate'].replace(',','', regex = True)
raw = pd.merge(raw, poverty, on='fips')
raw['poverty_estimate'] = raw['poverty_estimate'].astype(float)
raw.head()


,id_str,created_at,follower_count,friends_count,statefips,countyfips,countyname,is_urban,ogtext,cleantext,...,category,is_neutral,is_neutral2,is_positive2,is_negative2,is_neutral3,is_positive3,is_negative3,in_cdc,poverty_estimate
0,id:1226763665036075008,2020-02-10 07:03:44+00:00,270.0,539.0,32.0,3.0,Clark,1,having a mocha and avoiding coronavirus at the...,have mocha avoid coronavirus airport,...,POS,0,0,1,0,0,1,0,0,13.3
1,id:1226773799342706689,2020-02-10 07:44:00+00:00,312.0,125.0,32.0,3.0,Clark,1,sadly nothing will be done to help the people ...,sadly help people china communist government u...,...,NEG,0,0,0,1,0,0,1,0,13.3
2,id:1226327068389998592,2020-02-09 02:08:51+00:00,2887.0,831.0,32.0,3.0,Clark,1,the world is grieving for the wuhan doctor who...,world grieving wuhan doctor try warn colleague...,...,NEG,0,0,0,1,0,0,1,0,13.3
3,id:1233697011758587907,2020-02-29 10:14:23+00:00,7.0,169.0,32.0,3.0,Clark,1,risk of global coronavirus spread very high wa...,risk global coronavirus spread high warn china...,...,NEG,0,0,0,1,0,0,1,0,13.3
4,id:1233697205405437953,2020-02-29 10:15:09+00:00,7.0,169.0,32.0,3.0,Clark,1,coronavirus what are the chances of dying,coronavirus chances die,...,NEG,0,0,0,1,0,0,1,0,13.3


In [3]:
keywords = ['social distancing', 'social distance', 'physical distance', '6 feet', 'stay at home', 'school isolation', 'isolation', 'stay home', 'avoid touching', 'mask', 'covering', 'face shield', 'wear a mask', 'surgical mask', 'N95 respirator', 'wearing gloves', 'face shields', 'facial covering', 'skin protection', 'eye protection', 'ppe', 'wash hands', 'hand sanitizer', 'disinfect', 'clean', 'detergent', 'handwashing', 'hand hygiene', 'prevention hygiene', 'sprays', 'concentrates', 'wipes', 'routine cleaning', 'bleach solution', 'test', 'business closure']

topics = []
indexes = {i:0 for i in range(len(raw.index))}

for i, topic in enumerate(keywords):
    topics.append([1 if topic in text else 0 for text in raw['ogtext']])

for i, kw in enumerate(topics):
    for data_ind, value in enumerate(topics[i]):
        indexes[data_ind] += value

raw['in_cdc'] = [indexes[i] if 0 <= indexes[i] <= 1 else 1 for i in indexes]

cdc = raw[raw['in_cdc'] == 1]

print(f'there are a total of {len(cdc.index)} tweets after subsetting for cdc keywords')

there are a total of 53215 tweets after subsetting for cdc keywords


In [12]:
counties = raw.groupby(['fips']).agg({'is_negative':'mean', 'is_positive':'mean', 'is_neutral':'mean', 'poverty_estimate':'mean', 'state_name':'max', 'count':'count'}).reset_index()

counties = counties[counties['count'] > 15]
print(f'n counties {len(counties.index)}')
# print(counties.corr()**2)

import scipy

cats = 'negative positive neutral'.split(' ')

for cat in cats:
    x= counties[f'is_{cat}']
    y= counties['poverty_estimate']

    print(f'{cat}:   r                  p')
    print(scipy.stats.pearsonr(x, y))    # Pearson's r
    print()

n counties 909
negative:   r                  p
(0.046101683642716614, 0.16490135669612332)

positive:   r                  p
(-0.08355731884828414, 0.011730025207650648)

neutral:   r                  p
(0.05741652104513095, 0.08360667388703547)



In [13]:
counties = cdc.groupby(['fips']).agg({'is_negative':'mean', 'is_positive':'mean', 'is_neutral':'mean', 'poverty_estimate':'mean', 'state_name':'max', 'count':'count'}).reset_index()

counties = counties[counties['count'] > 15]
print(f'n counties {len(counties.index)}')

# print(counties.corr()**2)

# import scipy

cats = 'negative positive neutral'.split(' ')

for cat in cats:
    x= counties[f'is_{cat}']
    y= counties['poverty_estimate']

    print(f'{cat}:   r                  p')
    print(scipy.stats.pearsonr(x, y))    # Pearson's r
    print()

n counties 413
negative:   r                  p
(-0.026115491044656637, 0.596658951003852)

positive:   r                  p
(-0.0038761374518536656, 0.9374033108847634)

neutral:   r                  p
(0.046038058928351465, 0.350684115711116)

